In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import keras
import os
import h5py
import numpy as np
import pickle
import pandas as pd


In [ ]:
#print(M_1.shape)
# /content/drive/MyDrive/Recordings/Validation/Final_val.csv

# Reading validation file. 
df=pd.read_csv('/content/drive/MyDrive/Recordings/Validation/Final_val.csv')

# Reading training file.
df_train=pd.read_csv('/content/drive/MyDrive/Recordings/Train/Final-3.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Recordings/Test/final_test.csv')

df = df.drop(['Unnamed: 0', 'Video', 'PART.1', 'PART.2','TIMESTAMP', 'LANGUAGE', 'COUNTRY' , 'EDUCATION'], axis = 1)
df_train = df_train.drop(['Unnamed: 0', 'Video', 'PART.1', 'PART.2', 'LANGUAGE'], axis = 1)

list_id = df.ID_y
df_train.columns

Index(['ID_y', 'GENDER', 'AGE', 'OPENMINDEDNESS_Z', 'CONSCIENTIOUSNESS_Z',
       'EXTRAVERSION_Z', 'AGREEABLENESS_Z', 'NEGATIVEEMOTIONALITY_Z',
       'SESSION1', 'SESSION2', 'SESSION3', 'SESSION4'],
      dtype='object')

## Code for preparing data to feed to network.

This is dynamic. So if you feed a dataframe, it will check the following:

a. It will check if each session files are present or not. If not, it loads zeros as data.  
b. It will check if face landmarks are there or not. If not, it loads zeros.  
c. It checks if the OCEAN labels are present or not. If not, it loads 1's

In [ ]:
temp = []

In [ ]:
df_train.head()

,ID_y,GENDER,AGE,OPENMINDEDNESS_Z,CONSCIENTIOUSNESS_Z,EXTRAVERSION_Z,AGREEABLENESS_Z,NEGATIVEEMOTIONALITY_Z,SESSION1,SESSION2,SESSION3,SESSION4
0,2,F,29,-1.683690,1.292321,-0.231237,1.160066,-0.312424,/content/drive/MyDrive/Annotations/Animals/002...,/content/drive/MyDrive/Annotations/Talk/Annota...,/content/drive/MyDrive/Annotations/Lego/002003...,/content/drive/MyDrive/Annotations/Ghost/00200...
1,3,F,28,-0.263908,0.616152,0.243083,0.982578,-0.437972,/content/drive/MyDrive/Annotations/Animals/002...,/content/drive/MyDrive/Annotations/Talk/Annota...,/content/drive/MyDrive/Annotations/Lego/002003...,/content/drive/MyDrive/Annotations/Ghost/00200...
2,3,F,28,-0.263908,0.616152,0.243083,0.982578,-0.437972,/content/drive/MyDrive/Annotations/Animals/003...,/content/drive/MyDrive/Annotations/Talk/Annota...,/content/drive/MyDrive/Annotations/Lego/003005...,/content/drive/MyDrive/Annotations/Ghost/00300...
3,4,M,31,-0.421661,-0.330485,-0.705557,0.272626,-0.312424,/content/drive/MyDrive/Annotations/Animals/004...,/content/drive/MyDrive/Annotations/Talk/Annota...,/content/drive/MyDrive/Annotations/Lego/004096...,/content/drive/MyDrive/Annotations/Ghost/00409...
4,4,M,31,-0.421661,-0.330485,-0.705557,0.272626,-0.312424,/content/drive/MyDrive/Annotations/Animals/004...,/content/drive/MyDrive/Annotations/Talk/Annota...,/content/drive/MyDrive/Annotations/Lego/004115...,/content/drive/MyDrive/Annotations/Ghost/00411...


In [ ]:
maleidx = []
femaleidx = []

def prepareData(df):  
  male = []
  female = []
  maleY = []
  femaleY = []
  total_landmarks = []
  data = {'Video':[], 'ID_y':[], 'minute': [], 'session': [], 'gender': [], 'OPENMINDEDNESS_Z':[], 'CONSCIENTIOUSNESS_Z':[],
      'EXTRAVERSION_Z':[], 'AGREEABLENESS_Z':[], 'NEGATIVEEMOTIONALITY_Z':[]}
  df_temp = pd.DataFrame(data)
  
  OCEANvals = ['OPENMINDEDNESS_Z', 'CONSCIENTIOUSNESS_Z',
      'EXTRAVERSION_Z', 'AGREEABLENESS_Z', 'NEGATIVEEMOTIONALITY_Z']
  
  face_zero_array = np.zeros(204)
  body_zero_array = np.zeros(72)
  hand_zero_array = np.zeros(63)
  zero_array = np.zeros(603000)

  sessions = ['SESSION1','SESSION2','SESSION3','SESSION4']
  landmarks_for_sess = []

  for index, row in df.iterrows():
      # Going through each ID's h5py file
        print(row['ID_y'])
        for session in sessions:
          # print(session)
          # print(row[session])

          # Check if the file exists
          if not os.path.isfile(row[session]):
            print("INACCESSIBLE ", row[session])
            continue
          # Landmarks for current session stored here

          # If it exists, go through it
          with h5py.File(row[session]) as f:
            ls= list(f.keys())
            frame = 1; 
            print(len(ls))
            current_minute = []
            
            # for each frame in the list of frames, till 1500
            for fr in ls:
              # if frame == 1500:
              #   break
              lndmrk = []
              # If no landmarks for current frame, then skip
              if len(f[fr])==0 :
                # If there are no landmarks, fill it with zero to maintain consistency, 402 because 402 features
                lndmrk.append(np.zeros(276))
                # print("Skipping this")
                frame = frame+1
                continue

              # if face landmarks are there
              if not len(f[fr]['face'])==0:
                face_landmarks = np.asarray(f[fr]['face']['landmarks'])
                face_landmarks = face_landmarks.reshape([1,-1])
                lndmrk = lndmrk+ list(face_landmarks[0])
                
              else:
                
                # if not there
                lndmrk = lndmrk+ list(face_zero_array)
              

              # If body landmarks are there
              if not len(f[fr]['body'])==0:
                body_landmarks = np.asarray(f[fr]['body']['landmarks'])
                body_landmarks = body_landmarks.reshape([1,-1])
                lndmrk = lndmrk+ list(body_landmarks[0])

              else: 

                # if not there
                lndmrk= lndmrk+ list(body_zero_array)
                
            #   COMMENTED OUT TO NOT CONSIDER HAND
              # # If hand landmarks are not there at all.
              # if len(f[fr]['hands']) == 0:
              #   lndmrk = lndmrk+ list(hand_zero_array)+list(hand_zero_array)
              #   # print("NO HAND LANDMARKS")
              # else:
              #   # if left hands landmarks are there, 
              #   if not (len(f[fr]['hands']['left']) == 0):
              #     left_hands_landmarks = np.asarray(f[fr]['hands']['left']['landmarks'])                  
              #     left_hands_landmarks = left_hands_landmarks.reshape([1,-1])
              #     lndmrk = lndmrk+ list(left_hands_landmarks[0])
              #   else:

              #     # if left landmarks not there
              #     lndmrk = lndmrk+ list(hand_zero_array)
                  
              #   # if right hands landmarks are there
              #   if not (len(f[fr]['hands']['right'])==0):
              #     right_hands_landmarks = np.asarray(f[fr]['hands']['right']['landmarks'])
              #     right_hands_landmarks = right_hands_landmarks.reshape(1,-1)
              #     lndmrk= lndmrk+ list(right_hands_landmarks[0])
              #   else:
              #     # if right not there
              #     lndmrk= lndmrk+ list(hand_zero_array)
                  
              
              # We get lndmrk as a list of (1,whatever)
              lndmrk = np.array(lndmrk)
              # Just to check
              # print( len(lndmrk))

              # Appending it to the session landmarks
              current_minute.append(lndmrk)
              frame = frame+1
              # print(frame)
              if frame%1500 == 0:
                minute = frame/1500
                vid_data = row[session].split('/') 
                video_name = vid_data[8]
                session_name = vid_data[7]
                personID = row['ID_y']
                # print(str(video_name)+' '+str(personID)+' '+str(session_name)+' '+str(minute))
                
                dat = {'Video':video_name, 'ID_y':personID, 'minute': minute, 'session': session_name, 'OPENMINDEDNESS_Z':row['OPENMINDEDNESS_Z'],
                       'CONSCIENTIOUSNESS_Z': row['CONSCIENTIOUSNESS_Z'],
                        'EXTRAVERSION_Z' : row['EXTRAVERSION_Z'],
                        'AGREEABLENESS_Z' : row['AGREEABLENESS_Z'],
                        'NEGATIVEEMOTIONALITY_Z' : row['NEGATIVEEMOTIONALITY_Z']}
                        
                df_temp = df_temp.append(dat, ignore_index=True)


                mean_minute = np.array(np.mean(current_minute, axis=0))
                # print(mean_minute)
                std_minute = np.array(np.std(current_minute, axis=0))
                minute_data = np.concatenate([mean_minute,std_minute])
                landmarks_for_sess.append(minute_data)


                

          print("Done with", session)

          print("landmarks shape = ",len(landmarks_for_sess),len(landmarks_for_sess[0]))
          # total_landmarks.append(landmarks_for_sess)
          # print(len(total_landmarks))
          print(len(df_temp.index))
          print(df_temp.tail())

  landmarks_for_sess = np.array(landmarks_for_sess)

  df_temp = pd.concat([df_temp, pd.DataFrame(landmarks_for_sess)], axis=1)
  
  # if the labels are there
  # list_cols = list(df_temp.columns)
  # print(list_cols)
  return df_temp

## defining lists for loading the data



In [ ]:
df_temp_train = prepareData(df_train)


2
30500
Done with SESSION1
landmarks shape =  20 552
20
                   Video  ID_y  ...  AGREEABLENESS_Z NEGATIVEEMOTIONALITY_Z
15  annotations_raw.hdf5   2.0  ...         1.160066              -0.312424
16  annotations_raw.hdf5   2.0  ...         1.160066              -0.312424
17  annotations_raw.hdf5   2.0  ...         1.160066              -0.312424
18  annotations_raw.hdf5   2.0  ...         1.160066              -0.312424
19  annotations_raw.hdf5   2.0  ...         1.160066              -0.312424

[5 rows x 10 columns]
8343


KeyboardInterrupt: ignored

In [ ]:
df_temp_train.to_csv("/content/drive/MyDrive/Recordings/Train/train_data.csv")